In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# place all *.py files in the same folder
import lib2
from lib2 import Predicate
from models import customXGB
from frequent_itemsets import preprocessDataset, runApriori, aprioriout2predicateList
from parameters import ParameterProxy

import matplotlib.pyplot as plt

In [2]:
DATAFILE = '../adult.data' # location of dataset
random_state = None # change to something for exactly reproducible results
sensitive_attribute = "Sex"
target_name = "label"
positive_label = ">50K"
negative_label = "<=50K"
model_train_fraction = 0.7

In [3]:
# specify feature names
feature_names = [
   "Age", "Workclass", "fnlwgt", "Education",
   "Education-Num", "Marital Status", "Occupation",
   "Relationship", "Race", "Sex", "Capital Gain",
   "Capital Loss", "Hours per week", "Country", "label"
]
# specify categorical columns
cate_columns = ['Workclass', 'Education', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']

In [4]:
# WARNING: after changing any of the values, restart the notebook

# define featureCost as a mapping holding, for each feature name, the respective cost of not keeping that feature constant
# Any feature not specified will have its cost set to 1
featureCosts = {"Sex": 100}

# define featureChange as a mapping from str to function, holding, for each feature name, the respective 
# function that calculates the cost of change from one value to another.
# Any feature change not specified will be set to 1 if there is change, and 0 otherwise.
def age_cost(age1: str, age2: str) -> int:
    return abs(int(age1) - int(age2))
featureChange = {"Age": age_cost}

# set the weights that manage the relative influence of coverage, correctness, feature cost and feature change
# in the objective function of the algorithm
l_cover = 1
l_correct = 2
l_cost = 1
l_change = 1

In [5]:
params = ParameterProxy(
    featureCosts=featureCosts,
    featureChanges=featureChange,
    lambda_correctness=l_correct,
    lambda_cover=l_cover,
    lambda_featureChange=l_change,
    lambda_featureCost=l_cost
)

# Data loading

Loads the dataset into variable `data`. Just run it.

In [6]:
data = pd.DataFrame(
  np.genfromtxt(DATAFILE, delimiter=', ', dtype=str),
  columns=feature_names
)

# Train test split

Split into train-test. The train set is used specifically for training the model, and nothing further.

In [7]:
X = data.drop(target_name, axis=1)
y = data[target_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=model_train_fraction, random_state=random_state)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Definition and Training of a black-box model

We use a black box model based on gradient boosted decision trees.

In [8]:
model = customXGB(n_estimators=300, max_depth=5)
model.fit(X_train, y_train, cate_columns=cate_columns)
model.predict(X_test.iloc[:100, :])

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

0 denotes the negative and 1 the positive class.

# Testing the model

We check if the model is satisfactory.

In [9]:
preds = model.predict(X_test)
print(classification_report(y_test.map({negative_label: 0, positive_label: 1}), preds))

              precision    recall  f1-score   support

           0       0.90      0.93      0.91      7420
           1       0.75      0.66      0.70      2349

    accuracy                           0.86      9769
   macro avg       0.82      0.79      0.81      9769
weighted avg       0.86      0.86      0.86      9769



# Finding the affected

We check the predictions of the model on the test data, i.e. our population. The ones that get a 0 (negative outcome) are called the affected individuals.

In [10]:
X_aff_idxs = np.where(model.predict(X_test) == 0)[0]
print(f"Test data shape: {X_test.shape}")
X_aff = X_test.iloc[X_aff_idxs, :]
print(f"Affected shape: {X_aff.shape}")
# X_aff.reset_index(inplace=True, drop=True)

Test data shape: (9769, 14)
Affected shape: (7704, 14)


This means that the model classifies into the negative class about 7000 out of the 10000 total (test set) individuals.

## and the unaffected

In [11]:
X_unaff_idxs = np.where(model.predict(X_test) == 1)[0]
X_unaff = X_test.iloc[X_unaff_idxs, :]
print(f"Unaffected shape: {X_unaff.shape}")
# X_aff.reset_index(inplace=True, drop=True)

Unaffected shape: (2065, 14)


In [12]:
X_unaff["Capital Gain"].value_counts()

0        1557
15024      96
7688       90
7298       75
99999      51
3103       35
5178       24
4386       23
8614       17
27828      16
20051      14
10520      10
14344      10
4787       10
9386        8
13550       7
14084       6
25236       4
7430        2
10566       2
15831       2
4931        1
25124       1
41310       1
15020       1
10605       1
4508        1
Name: Capital Gain, dtype: int64

# Running apriori adaptation

We now generate the frequent itemsets of the datasets. These are used by the global counterfactual generating algorithm, in order to try and cover as many affected individuals as possible.

Here, we have implemented a simple `runApriori` wrapper function, which basically runs the fpgrowth algorithm for frequent itemset mining of the library mlxtend. It returns a dataframe with two columns, an $itemset$ column which contains the itemsets, in the form value tuples, and a $support$ column, which contains the relative frequency with which the itemset is contained in the dataset. Meaning, the fraction of individuals which have this specific combination of feature values.

Notice that we can give a minimum support as an argument to the function. This means that it returns only those itemsets whose support are above this value.

*Note*: You can ignore the "+feature_name" part. It has been appended to every value for implementation reasons, so that we know the "type" of each value, i.e. the feature it corresponds to. For example, whether a 0 is a value for "Capital Loss" or for "Capital Gain".

In [13]:
males_affected = X_aff[X_aff["Sex"] == "Male"].drop([sensitive_attribute], axis=1)
males_unaffected = X_unaff[X_unaff["Sex"] == "Male"].drop([sensitive_attribute], axis=1)
females_affected = X_aff[X_aff["Sex"] == "Female"].drop([sensitive_attribute], axis=1)
females_unaffected = X_unaff[X_unaff["Sex"] == "Female"].drop([sensitive_attribute], axis=1)

In [14]:
freq_ma = runApriori(preprocessDataset(males_affected), min_support=0.03)
freq_mu = runApriori(preprocessDataset(males_unaffected), min_support=0.03)
freq_fa = runApriori(preprocessDataset(females_affected), min_support=0.03)
freq_fu = runApriori(preprocessDataset(females_unaffected), min_support=0.03)

In [15]:
RL_ma, _ = aprioriout2predicateList(freq_ma)
RL_mu, _ = aprioriout2predicateList(freq_mu)
RL_fa, _ = aprioriout2predicateList(freq_fa)
RL_fu, _ = aprioriout2predicateList(freq_fu)

In [16]:
d = X_test.drop([sensitive_attribute], axis=1)
freq_itemsets = runApriori(preprocessDataset(d), min_support=0.03)
freq_itemsets.reset_index()
# print(freq_itemsets.head())
# print(freq_itemsets.head(100).to_string())
RL, _ = aprioriout2predicateList(freq_itemsets)
# pprint(RL[:10])
# print(len(RL))

Next, we use the function `aprioriout2predicateList`, which "casts" the output of the frequent itemset mining algorithm to our internal representation of a "triple" (as in the ares paper). This representation is the class `Predicate`.

RL is the initial set of candidate predicates (taken as the output of the itemset algo), from which we will then pick pairs to represent our rules (as in ares paper).

# Running the optimization procedure

First, just turn the user-defined SD (subgroup descriptors) to predicates.

In [17]:
from metrics import incorrectRecoursesSubmodular, incorrectRecourses, cover, featureCost, featureChange
from optimization import optimize_vanilla
from lib2 import recourse_report
from recourse_sets import TwoLevelRecourseSet

In [18]:
SD = list(map(Predicate.from_dict, [
    {sensitive_attribute: val} for val in data[sensitive_attribute].unique()
]))

In [19]:
print(X_aff.shape)

(7704, 14)


Now, we run the submodular optimization.

In [21]:
%%time

final_rules = optimize_vanilla(SD, RL[:400], X_aff[:400], model)

Total triples = 144
Calculated incorrect recourse for each triple
Calculated feature costs for each triple
Calculated feature changes for each feature
Calculated covers for each triple
CPU times: total: 18.4 s
Wall time: 2.21 s


In [22]:
two_level_recourses = TwoLevelRecourseSet.from_triples(final_rules[0])
print(recourse_report(two_level_recourses, X_aff, model))

Total coverage: 56.140% (over all affected).
Total incorrect recourses: 85.341% (over all those covered).
Total feature cost: 3.
Total feature change: 3.
If Sex = Female:
	If Marital Status = Never-married,
	Then Marital Status = Married-civ-spouse.
		Coverage: 17.601% over all affected.
		Incorrect recourses additive: 92.404% over all individuals covered by this rule.
		Incorrect recourses at-least-one: 92.404% over all individuals covered by this rule.
If Sex = Male:
	If Education-Num = 10,
	Then Education-Num = 13.
		Coverage: 13.318% over all affected.
		Incorrect recourses additive: 85.965% over all individuals covered by this rule.
		Incorrect recourses at-least-one: 85.965% over all individuals covered by this rule.
	If Education-Num = 9,
	Then Education-Num = 13.
		Coverage: 25.221% over all affected.
		Incorrect recourses additive: 80.082% over all individuals covered by this rule.
		Incorrect recourses at-least-one: 80.082% over all individuals covered by this rule.



In [23]:
from optimization import optimize

In [24]:
print(len(RL_ma))
print(len(RL_mu))
print(len(RL_fa))
print(len(RL_fu))

6395
8739
6605
9894


In [28]:
%%time

final_rules_separate = optimize(
    SD,
    ifs={"Male": RL_ma[:500], "Female": RL_fa[:500]},
    thens={"Male": RL_mu[:500], "Female": RL_fu[:500]},
    X_aff=X_aff,
    model=model
)

Total triples = 218
Calculated incorrect recourse for each triple
Calculated feature costs for each triple
Calculated feature changes for each feature
Calculated covers for each triple
CPU times: total: 2min 53s
Wall time: 1min 35s


In [29]:
two_level_recourses = TwoLevelRecourseSet.from_triples(final_rules_separate[0])
print(recourse_report(two_level_recourses, X_aff, model))

If Sex = Female:
	If Marital Status = Never-married,
	Then Marital Status = Married-civ-spouse.
		Coverage: 18.446% over all affected.
		Incorrect recourses additive: 92.862% over all individuals covered by this rule.
		Incorrect recourses at-least-one: 92.862% over all individuals covered by this rule.
	If Marital Status = Divorced,
	Then Marital Status = Married-civ-spouse.
		Coverage: 10.249% over all affected.
		Incorrect recourses additive: 87.280% over all individuals covered by this rule.
		Incorrect recourses at-least-one: 87.280% over all individuals covered by this rule.
If Sex = Male:
	If Education-Num = 9,
	Then Education-Num = 13.
		Coverage: 23.816% over all affected.
		Incorrect recourses additive: 76.152% over all individuals covered by this rule.
		Incorrect recourses at-least-one: 76.152% over all individuals covered by this rule.
	If Education-Num = 10,
	Then Education-Num = 13.
		Coverage: 13.605% over all affected.
		Incorrect recourses additive: 84.630% over all i